# 1. TF 함수와 콘크리트 함수

- 다형성 함수
- 여러 가지 타입의 입력 지원
- 콘크리트 함수 : 입력 조합에 특화된 그래프를 가진 함수

# 2. 함수 정의와 함수 그래프 탐험하기

- 콘크리트 함수 → 계산 그래프

# 3. 트레이싱 자세히 보기

- 심볼릭 텐서 → 파이썬 함수 트레이싱 될 때 실행

# 4. 오토그래프로 제어 흐름 표현하기

- 동적 반복문 → 직관적이지 않음
- 오토그래프 사용

# 5. TF 함수에서 변수와 다른 리소스 다루기

- 리소스 : 변수, 큐, 데이터셋과 같이 상태가 있는 객체
- 상태가 있는 연산 → 등장 순서대로 실행

# 6. 케라스로 TF 함수 사용하기

- 자동 TF 함수 변환
- 직접 작성한 코드 → TF 함수 변환 불가
- 코드 디버깅 → 즉시 실행 모드

In [1]:
import tensorflow as tf


@tf.function
def tf_cube(x):
    return x ** 3

In [2]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function

<ConcreteFunction (x: TensorSpec(shape=(), dtype=tf.float32, name=None)) -> TensorSpec(shape=(), dtype=tf.float32, name=None) at 0x1641CF3D0>

In [3]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [4]:
concrete_function.graph

In [6]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [7]:
pow_op = ops[2]
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [8]:
pow_op.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

In [9]:
concrete_function.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [11]:
concrete_function.graph.get_tensor_by_name('Identity:0')

<tf.Tensor 'Identity:0' shape=() dtype=float32>

In [13]:
concrete_function.function_def.signature

name: "__inference_tf_cube_7"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

In [15]:
@tf.function
def tf_cube(x):
    print(f"x=, {x}")
    return x ** 3

In [16]:
result = tf_cube(tf.constant(2.0))

x=, Tensor("x:0", shape=(), dtype=float32)


In [17]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [18]:
result = tf_cube(tf.constant(3.0))
result = tf_cube(tf.constant(4.0))

In [19]:
result = tf_cube(2)

x=, 2


In [20]:
result = tf_cube(3)

x=, 3


In [21]:
result = tf_cube(tf.constant([[1., 2.]]))

x=, Tensor("x:0", shape=(1, 2), dtype=float32)


In [22]:
result = tf_cube(tf.constant([[3., 4.], [5., 6.]]))

x=, Tensor("x:0", shape=(2, 2), dtype=float32)


In [23]:
result = tf_cube(tf.constant([[[7., 8.], [9., 10.]]]))

x=, Tensor("x:0", shape=(1, 2, 2), dtype=float32)


In [24]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    return images[:, ::2, ::2]

In [26]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])
preprocessed_images = shrink(img_batch_1)
preprocessed_images = shrink(img_batch_2)

In [27]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
preprocessed_images = shrink(img_batch_3)

TypeError: Binding inputs to tf.function failed due to `Can not cast TensorSpec(shape=(2, 2, 2), dtype=tf.float32, name=None) to TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)`. Received args: (<tf.Tensor: shape=(2, 2, 2), dtype=float32, numpy=
array([[[0.0862695 , 0.7483548 ],
        [0.91220474, 0.72628903]],

       [[0.17323399, 0.8117585 ],
        [0.40414655, 0.00920999]]], dtype=float32)>,) and kwargs: {} for signature: (images: TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).

In [28]:
@tf.function
def add_10(x):
    for i in range(10):
        x += 1
    return x

In [29]:
add_10(tf.constant(0))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [30]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

In [31]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

In [32]:
counter = tf.Variable(0)


@tf.function
def increment(counter, c=1):
    return counter.assign_add(c)


increment(counter)
increment(counter)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [34]:
function_def = increment.get_concrete_function(counter).function_def
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

In [35]:
counter = tf.Variable(0)


@tf.function
def increment(c=1):
    return counter.assign_add(c)

In [36]:
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0)
        
    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)